In [45]:
#TODO- write the outputs to netCDF4 files 
#    - check if input files exist
#    - check input file format
#    - check the time it starts (example if it start from mins not divisible by 5)

import pandas as pd
import numpy as np
from netCDF4 import Dataset
import xarray as xr
import glob
import os



def aggregateData(input_files_list,required):  
    
    if input_files_list:

        outputDF=pd.DataFrame() #empty dataframe to hold the output
        for i in range(len(input_files)): #for every file in the Input DATA folder
            df = (xr.open_dataset(input_files[i])).to_dataframe() #convert the netCDF4 data to a dataframe
            df5min=df[required].resample("5T").mean() #aggregating it to 5 mins interval
            outputDF=outputDF.append(df5min) #keep appending to the result data frame

        outputDF.rename(columns={'atmos_pressure':'atmospheric_pressure','rh_mean':'relative_humidity',
                         'temp_mean':'mean_temperature'}, inplace=True) #changing the column names of the dataframe to required format
        return outputDF
    else:
        return "Cannot find input files"


def writeCDFfile(output_file_name,output_file_path,outputDF):
    
    if not os.path.exists(output_file_path):
        os.makedirs(output_file_path)
    
    existing_output_files=glob.glob("output/*.cdf")
    
    if output_file_name in existing_output_files:
        return "Please rename your output file"
    else:
        f =Dataset(output_file_name,'w', format='NETCDF4') #'w' stands for write

        time=f.createDimension('time',len(outputDF))
        atmospheric_pressure=f.createVariable('atmospheric_pressure','f4','time')
        relative_humidity=f.createVariable('relative_humidity','f4','time')
        mean_temperature=f.createVariable('mean_temperature','f4','time')
        timeVar=f.createVariable('time','f8','time')

        atmospheric_pressure[:]=outputDF['atmospheric_pressure'].values
        relative_humidity[:]=outputDF['relative_humidity'].values
        mean_temperature[:]=outputDF['mean_temperature'].values
        timeVar[:]=outputDF.index.values #TODO- date time conversion problem. recheck definition of datatype and units
        f.close()
        return "Completed writing to output CDF file"





output_file_name='s1gpmetavgE13.b1.20180101.000000.cdf'
output_file_path='output'
input_files_list=glob.glob("DATA/*.cdf") #get the list of input file names with the extension cdf in the input "DATA" folder

required=['atmos_pressure','rh_mean','temp_mean'] #required columns to aggregate. If needed can add more column names to add in the output file

#FUNCTION CALLS
outputDF=aggregateData(input_files_list,required) #Calling the function to create an aggregated dataframe of all files
writeCDFfile(output_file_name,output_file_path,outputDF) #Calling function to write the output to the outputCDF

'Completed writing to output CDF file'

In [1]:
#import pip
#pip.main(['install','netCDF4'])
#pip.main(['install','xarray'])
#import netCDF4


###Write in a class format


#TODO- remove unneccessary packages
#from matplotlib import pyplot as plt
import pandas as pd
#import netCDF4 as netcdf
import numpy as np
from netCDF4 import Dataset
import xarray as xr
import glob

output_file_name='sgpmetavgE13.b1.20180101.000000.cdf'
input_files=glob.glob("DATA/*.cdf")
input_files

required=['atmos_pressure','rh_mean','temp_mean'] #required columns to aggregate. If needed can add more column names to add in the output file
outputDF=pd.DataFrame() #empty dataframe to hold the output
for i in range(len(input_files)): #for every file in the Input DATA folder
    df = (xr.open_dataset(input_files[i])).to_dataframe() #convert the netCDF4 data to a dataframe
    df5min=df[required].resample("5T").mean() #aggregating it to 5 mins interval
    outputDF=outputDF.append(df5min) #keep appending to the result data frame

outputDF.rename(columns={'atmos_pressure':'atmospheric_pressure','rh_mean':'relative_humidity',
                         'temp_mean':'mean_temperature'}, inplace=True) #changing the column names of the dataframe to required format
#outputDF.head()

#TODO- write the outputs to netCDF4 files 
#    - check if input files exist
#    - check input file format
#    - check the time it starts (example if it start from mins not divisible by 5)

f =Dataset(output_file_name,'w', format='NETCDF4') #'w' stands for write

time=f.createDimension('time',len(outputDF))
atmospheric_pressure=f.createVariable('atmospheric_pressure','f4','time')
relative_humidity=f.createVariable('relative_humidity','f4','time')
mean_temperature=f.createVariable('mean_temperature','f4','time')
timeVar=f.createVariable('time','f8','time')#,'f8','time')

atmospheric_pressure[:]=outputDF['atmospheric_pressure'].values
relative_humidity[:]=outputDF['relative_humidity'].values
mean_temperature[:]=outputDF['mean_temperature'].values


timeVar[:]=outputDF.index.values #TODO- date time conversion problem. check in the definition of datatype and units
#time.units = 'hours since 0001-01-01 00:00:00.0'
#date2num(dates,units=times.units,calendar=times.calendar)
#timedate2num(dates,units=times.units,calendar=times.calendar)
f.close()

df = (xr.open_dataset(output_file_name)).to_dataframe()
df.head()